# Entropy Pooling Portfolio Optimization
This Jupyter notebook and the associated Python files are intended for performing a Markovitz portfolio optimization using Entropy Pooling to incportporate information contained in user-specified return scenarios and views.

Credits: Lauri Jokinen, Christian Segercrantz and Ilmari Vauhkonen

### 1) Imports and setting parameters
The first step is to import all necessary packages and to specify all user-defined parameters.

In [1]:
import pandas as pd 
import numpy as np
import os

from plotnine import ggplot, geom_area, aes, labs, scale_x_continuous
from tqdm.notebook import tqdm 

# Import functions
from entropy_pooling import views
from entropy_pooling import entropy_minimizer
from entropy_pooling import markoviz_optimizer

In [2]:
### ALL PARAMETERS SPECIFIED BY USER ARE SET HERE ###

### Set the current working directory correctly
# directory = ''
# os.chdir(directory)

### Filenames (and sheet names if necessary) of the input Excel files
scenarios_file = 'data/data.xlsx'
views_file     = 'data/sample_portfolio2_views.xlsx'
views_sheet    = 2
deltas_file    = 'data/sample_portfolio2_deltas.xlsx'
constrs_file   = 'data/sample_portfolio2_constrs.xlsx'

### Other parameters to be defined by the user
confidence_weight = 1 # between [0,1], how big a 'compromise' is done betw. prior and posterior distributions
total_capital     = 417000000 # The total (net) amount of capital that is invested in the portfolio.
mu_0              = 0.0005 * total_capital # The minimum expected return of the portfolio (in Euros)
c                 = 1 #Probability of the posterior distribution happening
leave_out         = [13,14] #Which assets to leave out of the visualization markowitz model. Give as array of indices.

### 2) Uploading user inputs
We proceed by uploading the input data: factor scenarios, views, asset delta matrix and optimization constraints.

In [3]:
# Scenarios and prior probabilities
scenarios, prior = markoviz_optimizer.load_factor_scenarios(scenarios_file, scale_by_100 = True)
# Views
(A,b,C,d) = views.load(views_sheet_name = views_file, views_subsheet_name = views_sheet);
# Asset deltas
deltas, asset_names = markoviz_optimizer.load_asset_deltas(deltas_file)
# Portfolio constraints
constr_A, lb, ub = markoviz_optimizer.load_portfolio_constraints(constrs_file)

### 3) Entropy minimization
Now we minimize the relative entropy between the given prior distribution and the posterior distribution, so that the view(s), expressed in terms of linear constraints, is (are) satisfied.

In [4]:
posterior = entropy_minimizer.full_confidence_posterior(prior, A, b, C, d, verbose = True)
weighted_posterior = entropy_minimizer.confidence_weighted_posterior(prior, posterior, c)

Results
Optimal dual variable values:  [ 2.81186418 -0.97813745]
Jacobian matrix at optimum [-3.06266828e-06 -4.02891634e-07]


### 4) Computing the asset-specific return scenarios
This is a necessary step so that we can formulate the expected returns and variance-covariance matrix needed for the Markowitz optimization.

In [5]:
asset_scenarios = markoviz_optimizer.asset_scenarios(scenarios, deltas, asset_names)

In [9]:
### 5) The Markowitz optimization

optimal = markoviz_optimizer.optimizer(asset_scenarios,
                                         weighted_posterior,
                                         mu_0 = 0,
                                         manual_constraints = (constr_A, lb, ub),
                                         visualize = False,
                                         verbose = 1)
for i, col in enumerate(asset_scenarios.columns):
  print("{}: {}".format(col, np.round(optimal.x[i],decimals = 0)))

The optimization was succesful: True
Global Equities: 7098568.0
DM Equities: 7098570.0
EM Equities: 7098538.0
European Equities: 7098566.0
US Equities: 7098575.0
EUR Money Markets: 53253618.0
EUR Government Bonds: 86221436.0
EUR Covered Bonds: 53253596.0
EUR Investment Grade Corporate Bonds: 53253577.0
European High Yield Corporate Bonds: 41700000.0
EMD Hard Currency: 53253545.0
Hedge Funds: 20285771.0
Listed Private Equity: 20285640.0
Technical Provisions: 1.0
IR Hedges: 1.0
